QC of target data
input files -> merged_1_22_data.bed/bim/fam
platform- bash/r/python on jupyter notebook of UKB RAP

intermediate files -> snps_pass_info_all.txt (INFO score<0.8)

output files->

Step (1) Sample size

Step (2) Genome build

In [ ]:
#Step(3) Remove SNPs with info score <0.8
#python
!dx download "project-J0v3JFQJ7Vj18G11pBV099b6:/Bulk/Imputation/UKB imputation from genotype/*.mfi.txt"

#r
for (i in 1:22){
mfi_chr <- read.table(paste("ukb22828_c", i, "_b0_v3.mfi.txt", sep = ""), header = F)
names(mfi_chr) <- c("Alternate_id","RS_id","Position","Allele1","Allele2","MAF","Minor_Allele","Info_score")
snpsPassInfo <- mfi_chr[mfi_chr$Info_score > 0.8,]

#bash
cat snpsPassInfo_chr*.txt > snps_pass_info_all.txt

wc -l snps_pass_info_all.txt
#28941271 SNPs  (original 97059328 SNPs)

#python
!dx upload snps_pass_info_all.txt --path project-J0v3JFQJ7Vj18G11pBV099b6:/aung/





Install plink 

In [ ]:
#bash
wget https://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20210606.zip

#python
!unzip plink_linux_x86_64_20210606.zip
!chmod +x plink
!./plink --version

Download the files


In [ ]:
!dx download "project-J0v3JFQJ7Vj18G11pBV099b6:/aung/target_data/merged_1_22_data.bim"
!dx download "project-J0v3JFQJ7Vj18G11pBV099b6:/aung/target_data/merged_1_22_data.fam"
!dx download "project-J0v3JFQJ7Vj18G11pBV099b6:/aung/target_data/merged_1_22_data.bed"
!dx download "project-J0v3JFQJ7Vj18G11pBV099b6:/aung/snps_pass_info_all.txt"

In [ ]:
!./plink \
  --bfile merged_1_22_data \
  --extract snps_pass_info_all.txt \
  --make-bed \
  --out data_info08

Step(4) Remove missingness

In [ ]:
!./plink --bfile data_info08 --missing --out data_missing

In [ ]:
!./plink \
  --bfile data_info08 \
  --mind 0.01 \
  --make-bed \
  --out data_mind \
  --threads 16 \
  --memory 170000

In [ ]:
./plink \
  --bfile data_mind \
  --geno 0.01 \
  --make-bed \
  --out data_geno \
  --threads 16 \
  --memory 170000


ambiguous and multiletter alleles SNP IDs

In [ ]:
awk '($5 == "A" && $6 == "T") || ($5 == "T" && $6 == "A") || \
     ($5 == "C" && $6 == "G") || ($5 == "G" && $6 == "C") || \
     length($5) > 1 || length($6) > 1' \
     data_geno.bim > snps_remove_ambig_multiletter.txt

     

In [ ]:
awk '{print $2}' snps_remove_ambig_multiletter.txt > ambiguous_multiletter_snp_ids.txt

awk '{print $1, $1}' ambiguous_multiletter_snp_ids.txt > ambi_multilet_snp_ids.txt

sex mismatch sample IDs

In [ ]:
#r
#create sex_mismatch_ids.txt
# Load the participant IDs
ids <- read.csv("sex_mismatch_data_participant.csv", header = TRUE)

# Assuming the ID column is called "eid" (change if it's different)
ids_plink <- data.frame(FID = ids$eid, IID = ids$eid)

# Save in PLINK format
write.table(ids_plink, "sex_mismatch_ids.txt",
            quote = FALSE, row.names = FALSE, col.names = FALSE, sep = "\t")

ancestry/ caucasian control 


In [ ]:
eth<-read.csv("ethnicity.csv", header = TRUE)
eth$FID <- eth$eid
eth$IID <- eth$eid
eth_out <- eth[, c("FID", "IID")]
write.table(eth_out, "ethnicity_ids.txt", quote = FALSE, row.names = FALSE, col.names = FALSE, sep = "\t")

relatedness


In [ ]:
rel <- read.table("ukb_relate.dat", header = FALSE)
colnames(rel) <- c("ID1", "ID2", "HetHet", "IBSO", "Kinship")
related <- subset(rel, Kinship > 0.0884)
remove_ids <- unique(related$ID2)   # choose one ID from each pair
write.table(remove_ids, "remove_related.txt", col.names = FALSE, row.names = FALSE, quote = FALSE)


second_ids <- read.table("remove_related.txt", header = FALSE)
second_ids$V2 <- second_ids$V1  # duplicate as FID and IID
write.table(second_ids, "remove_related_formatted.txt", col.names = FALSE, row.names = FALSE, quote = FALSE)

#bash
tail -n +2 remove_related_formatted.txt > remove_related_noheader.txt


shift work

In [ ]:
shift<-read.csv("shift_work_participant.csv", header = TRUE)
shift$FID <- shift$eid
shift$IID <- shift$eid
shift_out <- shift[, c("FID", "IID")]
write.table(shift_out, "shift_worker_ids.txt", quote = FALSE, row.names = FALSE, col.names = FALSE, sep = "\t")

duplicate SNPs
fam <- read.table("data_geno.fam", header = FALSE)
colnames(fam) <- c("FID", "IID", "PID", "MID", "Sex", "Phenotype")
dup_ids <- fam[duplicated(fam$IID), ]

In [ ]:
!./plink \
  --bfile data_geno \
  --list-duplicate-vars ids-only suppress-first \
  --out dup_vars \
  --threads 32 \
  --memory 320000

In [ ]:
#dulpicate participants


heterozygosity

In [ ]:
!./plink \
  --bfile data_geno \
  --het \
  --out target_het \
  --threads 32 \
  --memory 320000


In [ ]:
#r
het <- read.table("target_het.het", header = TRUE)
mean_F <- mean(het$F, na.rm = TRUE)
sd_F <- sd(het$F, na.rm = TRUE)

# Define thresholds
lower_bound <- mean_F - 3 * sd_F
upper_bound <- mean_F + 3 * sd_F

# Flag outliers
het$outlier <- het$F < lower_bound | het$F > upper_bound

# View summary
table(het$outlier)


In [ ]:
#r
het_outliers <- subset(het, outlier == TRUE)
write.table(het_outliers[, c("FID", "IID")], "heterozygosity_outliers.txt",
            row.names = FALSE, col.names = FALSE, quote = FALSE, sep = "\t")

In [ ]:
!./plink \
  --bfile data_geno \
  --maf 0.005 \
  --hwe 1e-6 \
  --remove data_missing.nosex \
  --remove sex_mismatch_ids.txt \
  --remove remove_related_formatted.txt \
  --remove heterozygosity_outliers.txt \
  --remove shift_worker_ids.txt \
  --keep ethnicity_ids.txt \
  --exclude ambi_multilet_snp_ids.txt \
  --exclude dup_vars.dupvar \
  --make-bed \
  --out target_qc_data \
  --threads 32 \
  --memory 320000


In [ ]:
cat data_missing.nosex \
    sex_mismatch_ids.txt \
    remove_related_noheader.txt \
    heterozygosity_outliers.txt \
    shift_worker_ids.txt \
    | sort -u > all_remove_ids.txt


In [ ]:
awk '{print $1}' ambi_multilet_snp_ids.txt > ambi_multilet_snp_ids_singlecol.txt

cat ambi_multilet_snp_ids_singlecol.txt dup_vars.dupvar | sort -u > all_exclude_snps.txt

In [ ]:
!./plink \
  --bfile data_geno \
  --maf 0.005 \
  --hwe 1e-6 \
  --remove all_remove_ids.txt \
  --exclude all_exclude_snps.txt \
  --make-bed \
  --out target_qc_data \
  --threads 32 \
  --memory 320000


In [ ]:
#r
fam <- read.table("target_qc_data.fam", header = FALSE,
                  colClasses = c("NULL","NULL","NULL","NULL","integer","NULL"))
sex <- fam$V5

n_male    <- sum(sex == 1, na.rm = TRUE)
n_female  <- sum(sex == 2, na.rm = TRUE)
n_unknown <- sum(sex == 0 | is.na(sex))

cat(sprintf("Male: %d\nFemale: %d\nUnknown: %d\nTotal: %d\n",
            n_male, n_female, n_unknown, length(sex)))


Male: 182590
Female: 217973
Unknown: 0
Total: 400563

In [ ]:
# from the folder with target_qc_data.{bed,bim,fam}
!./plink --bfile target_qc_data --freq --out target_freq --threads 32 --memory 320000
# makes target_freq.frq with columns: CHR SNP A1 A2 MAF NCHROBS
# (here A1 = minor allele; MAF = freq of A1)


In [ ]:
library(data.table)

ss  <- fread("asd_cojo.txt")            # columns: SNP A1 A2 freq b SE P N
frq <- fread("target_freq.frq")
setnames(frq, c("CHR","SNP","MINOR","MAJOR","MAF","NCHROBS"))

# ensure consistent case
ss[,  c("A1","A2") := .(toupper(A1),  toupper(A2))]
frq[, c("MINOR","MAJOR") := .(toupper(MINOR), toupper(MAJOR))]

# merge and compute effect-allele frequency
ss <- merge(ss, frq[, .(SNP, MINOR, MAJOR, MAF)], by = "SNP", all.x = TRUE)

# if A1 (effect allele in ss) equals MINOR -> freq = MAF; if equals MAJOR -> freq = 1 - MAF
ss[, freq := fifelse(!is.na(MAF) & A1 == MINOR, MAF,
              fifelse(!is.na(MAF) & A1 == MAJOR, 1 - MAF, freq))]

# optional: write any allele-mismatch rows to inspect
bad <- ss[!is.na(MAF) & !(A1 %chin% c(MINOR, MAJOR))]
if (nrow(bad)) fwrite(bad, "allele_mismatch_to_target.tsv", sep = "\t")

# save with same columns/order
fwrite(ss[, .(SNP, A1, A2, freq, b, SE, P, N)],
       "asd_cojo.with_target_freq.txt", sep = "\t", quote = FALSE, na = "NA")